# MNE : From raw data to epochs and evoked responses (ERF/ERP)

`
Authors:
Alexandre Gramfort
Denis A. Engemann
`

In [2]:
%matplotlib qt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

First, load the _mne_ package:

In [3]:
import mne

We set the log-level to 'warning' so the output is less verbose

In [4]:
mne.set_log_level('warning')

### Remember if you need help just ask... the machine

In [6]:
mne.pick_types?

## Access raw data

You should have downloaded the `ds000117-practical` folder.

In [7]:
import os

# Change the following path to where the folder ds000117-practical is on your disk
data_path = os.path.expanduser("~/work/data/ds000117-practical/")

raw_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

In [8]:
print(raw_fname)

/Users/claire/work/data/ds000117-practical/derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif


In [9]:
ls $raw_fname

/Users/claire/work/data/ds000117-practical/derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif*


Read data from file:

In [10]:
mne.io.read_raw_fif?

In [12]:
raw = mne.io.read_raw_fif(raw_fname, preload=False)
print(raw)

<Raw  |  sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif, n_channels x n_times : 404 x 540100 (491.0 sec), ~7.4 MB, data not loaded>


Note the `preload=False` which states that no data is actually in memory.

For general info on importing MEG see:
https://mne.tools/stable/auto_tutorials/io/plot_10_reading_meg_data.html
or EEG see:
https://mne.tools/stable/auto_tutorials/io/plot_20_reading_eeg_data.html

Now let's look at the measurement info. It will give details about:

   - sampling rate
   - filtering parameters
   - available channel types
   - bad channels
   - etc.

In [14]:
mne.__version__

'0.20.dev0'

In [16]:
print(raw.info)

<Info | 27 non-empty fields
    acq_pars : str | 21833 items
    bads : list | 0 items
    ch_names : list | MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, ...
    chs : list | 404 items (GRAD: 204, MAG: 102, EEG: 74, STIM: 3, MISC: 12, CHPI: 9)
    comps : list | 0 items
    custom_ref_applied : NamedInt | 0 (FIFFV_MNE_CUSTOM_REF_OFF)
    description : str | 36 items
    dev_head_t : Transform | 3 items
    dig : list | 137 items (3 Cardinal, 5 HPI, 75 EEG, 54 Extra)
    events : list | 1 items
    experimenter : str | 3 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 1 items
    hpi_results : list | 1 items
    hpi_subsystem : dict | 2 items
    line_freq : float | 50.0
    lowpass : float | 356.3999938964844 Hz
    meas_date : tuple | 1941-03-22 11:04:14 GMT
    meas_id : dict | 4 items
    nchan : int | 404
    proc_history : list | 1 items
    proj_id : ndarray | 1 items
    proj_name : str | 11 items
    projs : list | 0 items
    sfr

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>How many channels do you have for each type of sensors?</li>
    <li>What is the sampling frequency?</li>
    <li>Have the data been filtered?</li>
    <li>What is the frequency of the line noise?</li>
    <li>Is there any bad channel?</li>
    </ul>
</div>

INSERT ANSWERS HERE

raw.info is just a dictionary:

In [17]:
isinstance(raw.info, dict)

True

So we can access its elements this way:

In [18]:
raw.info['sfreq']  # Sampling frequency

1100.0

In [19]:
raw.info['bads']  # list of marked bad channels

[]

In [20]:
raw.info['line_freq']

50.0

Next let's see what channels are present. It is available via the `raw.ch_names` attribute.

In [22]:
type(raw.ch_names)

list

In [21]:
raw.ch_names[:10]

['MEG0113',
 'MEG0112',
 'MEG0111',
 'MEG0122',
 'MEG0123',
 'MEG0121',
 'MEG0132',
 'MEG0133',
 'MEG0131',
 'MEG0143']

You can index it as a list

In [23]:
raw.ch_names[42]

'MEG0432'

In [26]:
raw.ch_names[:3]

['MEG0113', 'MEG0112', 'MEG0111']

Channel type of a specific channel

In [27]:
channel_type = mne.io.pick.channel_type(raw.info, 75)
print('Channel #75 is of type:', channel_type)

channel_type = mne.io.pick.channel_type(raw.info, 320)
print('Channel #320 is of type:', channel_type)

Channel #75 is of type: grad
Channel #320 is of type: eeg


Info contains all the details about the sensors (type, locations, coordinate frame etc.)

In [28]:
len(raw.info['chs'])

404

In [29]:
type(raw.info['chs'])

list

In [30]:
raw.info['chs'][0]

{'scanno': 1,
 'logno': 113,
 'kind': 1,
 'range': 1.9073486328125e-05,
 'cal': 3.250000046861601e-09,
 'coil_type': 3012,
 'loc': array([-0.1066    ,  0.0464    , -0.0604    , -0.01532829,  0.00619847,
        -0.99986327, -0.18597366, -0.98255992, -0.00331254, -0.98243302,
         0.185894  ,  0.016216  ]),
 'unit': 201,
 'unit_mul': 0,
 'ch_name': 'MEG0113',
 'coord_frame': 1 (FIFFV_COORD_DEVICE)}

In [31]:
raw.info['chs'][330]

{'scanno': 331,
 'logno': 25,
 'kind': 2,
 'range': 0.00030517578125,
 'cal': 0.00019999999494757503,
 'coil_type': 1,
 'loc': array([ 5.63842431e-02,  3.68367434e-02,  9.40217227e-02,  8.26010015e-04,
         1.14762366e-01, -2.10680366e-02,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]),
 'unit': 107,
 'unit_mul': 0,
 'ch_name': 'EEG025',
 'coord_frame': 4 (FIFFV_COORD_HEAD)}

In [34]:
raw.plot_sensors(kind='topomap', ch_type='grad');

In [35]:
raw.plot_sensors(kind='topomap', ch_type='eeg');

In [36]:
raw.info

<Info | 27 non-empty fields
    acq_pars : str | 21833 items
    bads : list | 0 items
    ch_names : list | MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, ...
    chs : list | 404 items (GRAD: 204, MAG: 102, EEG: 74, STIM: 3, MISC: 12, CHPI: 9)
    comps : list | 0 items
    custom_ref_applied : NamedInt | 0 (FIFFV_MNE_CUSTOM_REF_OFF)
    description : str | 36 items
    dev_head_t : Transform | 3 items
    dig : list | 137 items (3 Cardinal, 5 HPI, 75 EEG, 54 Extra)
    events : list | 1 items
    experimenter : str | 3 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 1 items
    hpi_results : list | 1 items
    hpi_subsystem : dict | 2 items
    line_freq : float | 50.0
    lowpass : float | 356.3999938964844 Hz
    meas_date : tuple | 1941-03-22 11:04:14 GMT
    meas_id : dict | 4 items
    nchan : int | 404
    proc_history : list | 1 items
    proj_id : ndarray | 1 items
    proj_name : str | 11 items
    projs : list | 0 items
    sfr

### Setting channel types

Some channels are wrongly defined as EEG in the file. 2 of these are EOG (EEG061 and EEG062) and EEG063 is actually an ECG channel. EEG064 was recording but not connected to anything, so we'll not make it `'misc'`. We will now set the channel types. This will be useful for automatic artifact rejection.

In [37]:
raw.set_channel_types?

In [38]:
raw.set_channel_types({'EEG061': 'eog',
                       'EEG062': 'eog',
                       'EEG063': 'ecg',
                       'EEG064': 'misc'})  # EEG064 free-floating el.

raw.rename_channels({'EEG061': 'EOG061',
                     'EEG062': 'EOG062',
                     'EEG063': 'ECG063'})

In [40]:
raw.info

<Info | 27 non-empty fields
    acq_pars : str | 21833 items
    bads : list | 0 items
    ch_names : list | MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, ...
    chs : list | 404 items (GRAD: 204, MAG: 102, EEG: 70, EOG: 2, ECG: 1, MISC: 13, STIM: 3, CHPI: 9)
    comps : list | 0 items
    custom_ref_applied : NamedInt | 0 (FIFFV_MNE_CUSTOM_REF_OFF)
    description : str | 36 items
    dev_head_t : Transform | 3 items
    dig : list | 137 items (3 Cardinal, 5 HPI, 75 EEG, 54 Extra)
    events : list | 1 items
    experimenter : str | 3 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 1 items
    hpi_results : list | 1 items
    hpi_subsystem : dict | 2 items
    line_freq : float | 50.0
    lowpass : float | 356.3999938964844 Hz
    meas_date : tuple | 1941-03-22 11:04:14 GMT
    meas_id : dict | 4 items
    nchan : int | 404
    proc_history : list | 1 items
    proj_id : ndarray | 1 items
    proj_name : str | 11 items
    projs : list |

In [41]:
raw.plot_sensors(kind='topomap', ch_type='eeg');

## Accessing the data

To access the data just use the [] syntax as to access any element of a list, dict etc.

In [42]:
start, stop = 0, 10
data, times = raw[:, start:stop]  # fetch all channels and the first 10 time points
print(data.shape)
print(times.shape)

(404, 10)
(10,)


In [43]:
times  # always starts at 0 by convention

array([0.        , 0.00090909, 0.00181818, 0.00272727, 0.00363636,
       0.00454545, 0.00545455, 0.00636364, 0.00727273, 0.00818182])

Note that `raw[]` returns both the data and the times array.

# Resampling the data

We will now change the sampling frequency of the data to speed up the computations.

In [44]:
raw.load_data()  # it is required to load data in memory
raw.resample(300)

<Raw  |  sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif, n_channels x n_times : 404 x 147300 (491.0 sec), ~461.4 MB, data loaded>

And let's remove the unecessary channels

In [45]:
raw.drop_channels?

In [46]:
to_drop = ['STI201', 'STI301', 'MISC201', 'MISC202', 'MISC203',
           'MISC204', 'MISC205', 'MISC206', 'MISC301', 'MISC302',
           'MISC303', 'MISC304', 'MISC305', 'MISC306', 'CHPI001',
           'CHPI002', 'CHPI003', 'CHPI004', 'CHPI005', 'CHPI006',
           'CHPI007', 'CHPI008', 'CHPI009']

In [47]:
raw.drop_channels(to_drop)

<Raw  |  sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif, n_channels x n_times : 381 x 147300 (491.0 sec), ~435.5 MB, data loaded>

In [49]:
raw.info

<Info | 27 non-empty fields
    acq_pars : str | 21833 items
    bads : list | 0 items
    ch_names : list | MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, ...
    chs : list | 381 items (GRAD: 204, MAG: 102, EEG: 70, EOG: 2, ECG: 1, MISC: 1, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : NamedInt | 0 (FIFFV_MNE_CUSTOM_REF_OFF)
    description : str | 36 items
    dev_head_t : Transform | 3 items
    dig : list | 137 items (3 Cardinal, 5 HPI, 75 EEG, 54 Extra)
    events : list | 1 items
    experimenter : str | 3 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 1 items
    hpi_results : list | 1 items
    hpi_subsystem : dict | 2 items
    line_freq : float | 50.0
    lowpass : float | 150.0 Hz
    meas_date : tuple | 1941-03-22 11:04:14 GMT
    meas_id : dict | 4 items
    nchan : int | 381
    proc_history : list | 1 items
    proj_id : ndarray | 1 items
    proj_name : str | 11 items
    projs : list | 0 items
    sfreq : f

# Visualizing raw data

See https://mne.tools/0.16/auto_tutorials/plot_visualize_raw.html
for more details.

Let's look at how to:
- browse data
- turn On/Off the PCA/SSP projections
- mark bad segments to obtained annotations
- group channel by types
- group channel by location

In [50]:
raw.plot?

In [51]:
%matplotlib qt

raw.plot();

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Do you see any bad channel?</li>
    <li>Do you see any bad segment of data?</li>
    <li>Do you see any more then EOG blinks?</li>
    </ul>
</div>

In [52]:
raw.annotations

<Annotations  |  1 segment : BAD_FLUX_JUMP (1), orig_time : 1941-03-22 11:04:14.988669>

In [53]:
raw.annotations.save('annot.csv')

In [54]:
!cat annot.csv

onset,duration,description
1941-03-22 11:08:22.697562,0.5860655737704974,BAD_FLUX_JUMP


### Filtering

In [56]:
raw.filter?

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Filter the raw data between 0Hz and 40Hz.</li>
    </ul>
</div>

In [57]:
raw.filter(0, 40)

<Raw  |  sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif, n_channels x n_times : 381 x 147300 (491.0 sec), ~435.5 MB, data loaded>

In [59]:
raw.filter?

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Plot the 10 first seconds of stimulation channel just using matplotlib.</li>
    </ul>
</div>

Tips:

- Pick the stim channel using `mne.pick_types`
- Get the data for this channel
- Plot it using `plt.plot`

In [82]:
# d = raw.get_data(picks=['EEG001', 'EEG002'])
# d = raw.get_data(picks=('eeg', 'mag'))
d = raw.get_data(picks=('grad',))
np.max(d)

2.385637307611377e-10

In [67]:
#### TODO
start = 0
stop = int(50 * raw.info['sfreq'])
data = raw.get_data('STI101', start=start, stop=stop)
data.shape

(1, 15000)

In [68]:
raw.times[start:stop].shape

(15000,)

In [72]:
plt.plot(raw.times[start:stop], data.T)

## Define and read epochs

First extract events:

In [85]:
events = mne.find_events(raw, stim_channel='STI101', verbose=True)

259 events found
Event IDs: [   5    6    7   13   14   15   17   18   19  256  261  262  263  269
  270  271  273  274  275 4096 4101 4102 4103 4109 4110 4111 4113 4114
 4115 4352]


<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>What is the type of the variable events?</li>
    <li>What is the meaning of the 3 columnes of events?</li>
    <li>How many events of type 5 do you see?</li>
    </ul>
</div>

In [92]:
# events(:, 3) == 5 # matlab
np.sum(events[:, 2] == 5)

25

There was a time offset of 34.5ms in the stimulus presentation. We need to correct events accordingly.

In [93]:
delay = int(round(0.0345 * raw.info['sfreq']))
events[:, 0] = events[:, 0] + delay

Let's visualize the paradigm:

In [94]:
events = events[events[:, 2] < 20] # take only events with code less than 20

In [95]:
fig = mne.viz.plot_events(events, raw.info['sfreq']);

For event trigger and conditions we use a Python dictionary with keys that contain "/" for grouping sub-conditions

In [96]:
event_id = {
    'face/famous/first': 5,
    'face/famous/immediate': 6,
    'face/famous/long': 7,
    'face/unfamiliar/first': 13,
    'face/unfamiliar/immediate': 14,
    'face/unfamiliar/long': 15,
    'scrambled/first': 17,
    'scrambled/immediate': 18,
    'scrambled/long': 19,
}

In [97]:
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'],
                          event_id=event_id);

In [100]:
raw.plot(event_id=event_id, events=events);

Define epochs parameters:

In [99]:
tmin = -0.5  # start of each epoch (500ms before the trigger)
tmax = 2.0  # end of each epoch (2000ms after the trigger)

Define the baseline period:

In [101]:
baseline = (-0.2, 0)  # means from 200ms before to stim onset (t = 0)

Define peak-to-peak (amplitude range) rejection parameters for gradiometers, magnetometers and EOG:

In [102]:
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)  # this can be highly data dependent

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li>The <a href="https://autoreject.github.io/">autoreject</a> project aims to solve this problem of reject parameter setting. See the <a href="https://www.sciencedirect.com/science/article/pii/S1053811917305013">paper</a>.</li>
    </ul>
</div>

In [103]:
# we are picky again, this time with EOG
picks = mne.pick_types(raw.info, meg=True, eeg=True, eog=True,
                       stim=False, exclude='bads')

In [104]:
picks

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

Extract epochs:

In [105]:
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks, baseline=baseline,
                    reject=reject)

In [106]:
print(epochs)

<Epochs  |   146 events (good & bad), -0.5 - 2 sec, baseline [-0.2, 0], ~7.4 MB, data not loaded,
 'face/famous/first': 25
 'face/famous/immediate': 10
 'face/famous/long': 14
 'face/unfamiliar/first': 25
 'face/unfamiliar/immediate': 12
 'face/unfamiliar/long': 10
 'scrambled/first': 25
 'scrambled/immediate': 14
 'scrambled/long': 11>


In [107]:
epochs.drop_bad()  # remove bad epochs based on reject

<Epochs  |   79 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~7.4 MB, data not loaded,
 'face/famous/first': 13
 'face/famous/immediate': 3
 'face/famous/long': 6
 'face/unfamiliar/first': 17
 'face/unfamiliar/immediate': 4
 'face/unfamiliar/long': 6
 'scrambled/first': 15
 'scrambled/immediate': 9
 'scrambled/long': 6>

In [108]:
epochs.load_data()  # load data in memory

<Epochs  |   79 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~178.5 MB, data loaded,
 'face/famous/first': 13
 'face/famous/immediate': 3
 'face/famous/long': 6
 'face/unfamiliar/first': 17
 'face/unfamiliar/immediate': 4
 'face/unfamiliar/long': 6
 'scrambled/first': 15
 'scrambled/immediate': 9
 'scrambled/long': 6>

Explore the epochs namespace

Hit ``epochs.<TAB>``

In [ ]:
# epochs.

See how epochs were dropped

In [109]:
epochs.plot_drop_log();

In [110]:
for drop_log in epochs.drop_log[:20]:
    print(drop_log)

[]
[]
[]
[]
[]
[]
[]
['EOG062']
['EOG062']
[]
['EOG062']
[]
[]
['EOG062']
[]
[]
[]
[]
[]
[]


In [111]:
epochs.copy().drop(10, reason="I don't like this one").plot_drop_log();

In [114]:
epochs.copy().drop(10, reason="I don't like this one").drop_log[:20]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 ['EOG062'],
 ['EOG062'],
 [],
 ['EOG062'],
 [],
 [],
 ['EOG062'],
 ["I don't like this one"],
 [],
 [],
 [],
 [],
 []]

In [115]:
raw.filter(0, 80)

<Info | 27 non-empty fields
    acq_pars : str | 21833 items
    bads : list | 0 items
    ch_names : list | MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, ...
    chs : list | 381 items (GRAD: 204, MAG: 102, EEG: 70, EOG: 2, ECG: 1, MISC: 1, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : NamedInt | 0 (FIFFV_MNE_CUSTOM_REF_OFF)
    description : str | 36 items
    dev_head_t : Transform | 3 items
    dig : list | 137 items (3 Cardinal, 5 HPI, 75 EEG, 54 Extra)
    events : list | 1 items
    experimenter : str | 3 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 1 items
    hpi_results : list | 1 items
    hpi_subsystem : dict | 2 items
    line_freq : float | 50.0
    lowpass : float | 40.0 Hz
    meas_date : tuple | 1941-03-22 11:04:14 GMT
    meas_id : dict | 4 items
    nchan : int | 381
    proc_history : list | 1 items
    proj_id : ndarray | 1 items
    proj_name : str | 11 items
    projs : list | 0 items
    sfreq : fl

In [120]:
events.shape

(146, 3)

In [119]:
epochs.events.shape

(79, 3)

In [123]:
events[epochs.selection] == epochs.events

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ T

### Wait a second, did we just loose half of our epochs due to EOG???

We can probably do better. Let's use the PCA-based signal space projection (SSP) to regress out spatial patterns related to EOG and other offenders, ie., ECG.

Here is the workflow, we'll first detect EOG artifacts and visualize their impact. Then we'll compute related spatial patterns to mitigate these artifacts.

In [124]:
# We can use a convenience function
eog_epochs = mne.preprocessing.create_eog_epochs(raw.copy().filter(1, None))
eog_epochs.average().plot_joint()

[<Figure size 800x420 with 7 Axes>,
 <Figure size 800x420 with 7 Axes>,
 <Figure size 800x420 with 7 Axes>]

In [129]:
raw.plot(events=eog_epochs.events);

Compare the y axes to the ERF/ERPs we just saw. We face important degrees of contamination!

In [125]:
projs_eog, _ = mne.preprocessing.compute_proj_eog(
    raw, n_mag=3, n_grad=3, n_eeg=3, average=True)

In [126]:
projs_eog

[<Projection  |  EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 204>,
 <Projection  |  EOG-planar--0.200-0.200-PCA-02, active : False, n_channels : 204>,
 <Projection  |  EOG-planar--0.200-0.200-PCA-03, active : False, n_channels : 204>,
 <Projection  |  EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>,
 <Projection  |  EOG-axial--0.200-0.200-PCA-02, active : False, n_channels : 102>,
 <Projection  |  EOG-axial--0.200-0.200-PCA-03, active : False, n_channels : 102>,
 <Projection  |  EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 70>,
 <Projection  |  EOG-eeg--0.200-0.200-PCA-02, active : False, n_channels : 70>,
 <Projection  |  EOG-eeg--0.200-0.200-PCA-03, active : False, n_channels : 70>]

In [127]:
layouts = [mne.find_layout(raw.info, ch_type=ch) for ch in ("eeg", "mag", "grad")]
mne.viz.plot_projs_topomap(projs_eog, layout=layouts);

Now the important question is how many components one should keep? Pro-tip: some of them don't look like clear artifact patterns. The good news is that we don't need to decide __*right*__ now.

In [130]:
# same business, same issue for ECG
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw.copy().filter(1, None))
ecg_epochs.average().plot_joint()

[<Figure size 800x420 with 7 Axes>,
 <Figure size 800x420 with 7 Axes>,
 <Figure size 800x420 with 7 Axes>]

We also face important insults from the cardiac signal... we'll project that out.

In [131]:
projs_ecg, _ = mne.preprocessing.compute_proj_ecg(
    raw, n_mag=3, n_grad=3, n_eeg=3, average=True)
mne.viz.plot_projs_topomap(projs_ecg, layout=layouts);

In [136]:
# now let's see how that would theoretically improve data preservation
reject2 = dict(mag=reject['mag'], grad=reject['grad']) 

epochs_clean = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
                          picks=picks, baseline=baseline,
                          preload=False,
                          reject=reject2)

epochs_clean.add_proj(projs_eog + projs_ecg)
#epochs_clean.copy().apply_proj().average().plot(spatial_colors=True);  # apply projs on a copy

<Epochs  |   146 events (good & bad), -0.5 - 2 sec, baseline [-0.2, 0], ~7.6 MB, data not loaded,
 'face/famous/first': 25
 'face/famous/immediate': 10
 'face/famous/long': 14
 'face/unfamiliar/first': 25
 'face/unfamiliar/immediate': 12
 'face/unfamiliar/long': 10
 'scrambled/first': 25
 'scrambled/immediate': 14
 'scrambled/long': 11>

In [137]:
epochs_clean.copy().average().plot(proj='interactive', spatial_colors=True);  # apply projs on a copy

now we keep all trials, probably we also removed some good signals.
we will postpone the selection of SSP vectors to later study the impact on
source localization

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li>MNE keeps SSP projections inside the info and allows to apply them later.</li>
    </ul>
</div>

In [138]:
# let's overwrite
epochs = epochs_clean

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Use ICA instead of SSP to remove artifacts</li>
    <li>What are potential benefits or disadvantages?</li>
    </ul>
</div>

### Visualization Epochs

See [this page](https://mne.tools/stable/auto_tutorials/epochs/plot_visualize_epochs.html) for options on how to visualize epochs.

Here is just an illustration to make a so-called ERP/ERF image:

In [140]:
raw.plot_psd(fmax=40);

In [141]:
epochs.plot_image(picks='EEG065', sigma=1.);

In [142]:
import matplotlib.pyplot as plt
plt.close('all')

In [143]:
epochs.plot();

### The epochs object is your MNE swiss army knife for processing segmented data!

- specialized methods for diagnostic plotting of data
- averaging
- saving
- manipulating data, e.g., rearranging or deleting single trials, resampling

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>How could you get the epochs corresponding to face?</li>
    <li>How could you get the epochs corresponding to a familiar face?</li>
    <li>How could you get the epochs corresponding to a scrambled face?</li>
    </ul>
</div>

In [149]:
epochs.event_id

{'face/famous/first': 5,
 'face/famous/immediate': 6,
 'face/famous/long': 7,
 'face/unfamiliar/first': 13,
 'face/unfamiliar/immediate': 14,
 'face/unfamiliar/long': 15,
 'scrambled/first': 17,
 'scrambled/immediate': 18,
 'scrambled/long': 19}

In [153]:
epochs[['unfamiliar', 'scrambled']]

<Epochs  |   97 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~7.6 MB, data not loaded,
 'face/unfamiliar/first': 25
 'face/unfamiliar/immediate': 12
 'face/unfamiliar/long': 10
 'scrambled/first': 25
 'scrambled/immediate': 14
 'scrambled/long': 11>

## basic IO 

The standard scenario is saving the epochs into .fif file together with all the header data.

In [154]:
epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')
epochs_fname

'/Users/claire/work/data/ds000117-practical/derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss-epo.fif'

In [155]:
epochs.save(epochs_fname, overwrite=True)  # note that epochs are save in files ending with -epo.fif

Overwriting existing file.
Loading data for 1 events and 751 original time points ...
Loading data for 145 events and 751 original time points ...


In [156]:
data = epochs.get_data()
data.shape

(145, 378, 751)

Scipy also supports reading and writing of matlab files. You can save your single trials with:

In [157]:
from scipy import io
epochs_data = epochs.get_data()
print(epochs_data.shape)
io.savemat('epochs_data.mat', dict(epochs_data=epochs_data),
           oned_as='row')

(145, 378, 751)


## Average the epochs to get ERF/ERP and plot it!

In [161]:
# refresh evoked
evoked = epochs.average()
evoked.del_proj()  # delete previous proj
# take first for each sensor type
evoked.add_proj(projs_eog[::3] + projs_ecg[::3])
evoked.apply_proj()  # apply

<Evoked  |  '0.17 * face/famous/first + 0.07 * face/famous/immediate + 0.10 * face/famous/long + 0.17 * face/unfamiliar/first + 0.08 * face/unfamiliar/immediate + 0.07 * face/unfamiliar/long + 0.17 * scrambled/first + 0.10 * scrambled/immediate + 0.08 * scrambled/long' (average, N=145), [-0.5, 2] sec, 376 ch, ~9.6 MB>

In [162]:
#evoked = epochs.average()
print(evoked)

<Evoked  |  '0.17 * face/famous/first + 0.07 * face/famous/immediate + 0.10 * face/famous/long + 0.17 * face/unfamiliar/first + 0.08 * face/unfamiliar/immediate + 0.07 * face/unfamiliar/long + 0.17 * scrambled/first + 0.10 * scrambled/immediate + 0.08 * scrambled/long' (average, N=145), [-0.5, 2] sec, 376 ch, ~9.6 MB>


In [163]:
plt.close('all')
evoked.plot(proj=True);

We can also show sensor position as line color:

In [164]:
evoked.plot(spatial_colors=True, proj=True);  # note the legend

In [ ]:
times = [0.0, 0.1, 0.18]
evoked.plot_topomap(ch_type='mag', times=times, proj=True);
evoked.plot_topomap(ch_type='grad', times=times, proj=True);
evoked.plot_topomap(ch_type='eeg', times=times, proj=True);

In [165]:
import numpy as np
# pure topography plots called topomap in the MNE jargon
for ch_type in ('mag', 'grad', 'eeg'):
    evoked.plot_topomap(times=np.linspace(0.05, 0.45, 8),
                        ch_type=ch_type, proj=True);

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>How does SSP impact the evoked responses? Use proj="interactive" to explore</li>
    </ul>
</div>

In [ ]:
# For example, let's say that we want to keep the first projs for now

# refresh evoked
evoked = epochs.average()
evoked.del_proj()  # delete previous proj
# take first for each sensor type
evoked.add_proj(projs_eog[::3] + projs_ecg[::3])
evoked.apply_proj()  # apply

Topoplot and time series can also be shown in one single plot:

In [166]:
evoked.plot_joint(times=[0.17]);

## Accessing and indexing epochs by condition

Epochs can be indexed by integers or slices to select a subset of epochs but also with strings to select by conditions `epochs[condition]`

In [167]:
epochs[0]  # first epoch

<Epochs  |   1 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~7.6 MB, data not loaded,
 'face/unfamiliar/first': 1>

In [168]:
epochs[:10]  # first 10 epochs

<Epochs  |   10 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~7.6 MB, data not loaded,
 'face/famous/first': 3
 'face/unfamiliar/first': 4
 'face/unfamiliar/immediate': 1
 'face/unfamiliar/long': 1
 'scrambled/first': 1>

In [169]:
epochs['face']  # epochs for a face

<Epochs  |   95 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~7.6 MB, data not loaded,
 'face/famous/first': 24
 'face/famous/immediate': 10
 'face/famous/long': 14
 'face/unfamiliar/first': 25
 'face/unfamiliar/immediate': 12
 'face/unfamiliar/long': 10>

In event_id, "/" selects conditions in a hierarchical way, e.g. here, "face" vs. "scrambled", "famous" vs. "unfamiliar", and MNE can select them individually

In [170]:
epochs['face'].average().\
    pick_types(meg='grad').crop(-0.1, 0.25).plot(spatial_colors=True);

Apply this to visualize all the conditions in `event_id`

In [171]:
plt.close('all')
for condition in ['face', 'scrambled']:
    epochs[condition].average().plot_topomap(times=[0.1, 0.15], title=condition);

## Write evoked data to disk

In [172]:
evoked_fname = raw_fname.replace('_meg.fif', '-ave.fif')
evoked_fname

'/Users/claire/work/data/ds000117-practical/derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss-ave.fif'

In [173]:
evoked.save(evoked_fname)  # note that the file for evoked ends with -ave.fif

or to write multiple conditions in 1 file

In [174]:
evokeds_list = [epochs[k].average() for k in event_id]  # get evokeds
mne.write_evokeds(evoked_fname, evokeds_list)

### Reading evoked from disk

It is also possible to read evoked data stored in a fif file:

In [175]:
evokeds_list = mne.read_evokeds(evoked_fname, baseline=(None, 0), proj=True)

Or give the explicit name of the averaged condition:

In [176]:
evoked1 = mne.read_evokeds(evoked_fname, condition="face/famous/first",
                           baseline=(None, 0), proj=True)

**Remark:** Did you notice that you can apply some preprocessing on reading the evokeds from disk?

### Compute a contrast:

In [177]:
evoked_face = epochs['face'].average()
evoked_scrambled = epochs['scrambled'].average()

In [178]:
contrast = mne.combine_evoked([evoked_face, evoked_scrambled], [0.5, -0.5])

Note that this combines evokeds taking into account the number of averaged epochs (to scale the noise variance)

In [179]:
print(evoked1.nave)  # average of 12 epochs
print(contrast.nave)  # average of 116 epochs

24
131.03448275862067


In [180]:
print(contrast)

<Evoked  |  '0.500 * 0.25 * face/famous/first + 0.11 * face/famous/immediate + 0.15 * face/famous/long + 0.26 * face/unfamiliar/first + 0.13 * face/unfamiliar/immediate + 0.11 * face/unfamiliar/long + -0.500 * 0.50 * scrambled/first + 0.28 * scrambled/immediate + 0.22 * scrambled/long' (average, N=131.03448275862067), [-0.5, 2] sec, 376 ch, ~9.8 MB>


In [182]:
fig = contrast.copy().pick('grad').crop(-0.1, 0.3).plot_joint()

In [183]:
evoked_face

<Evoked  |  '0.25 * face/famous/first + 0.11 * face/famous/immediate + 0.15 * face/famous/long + 0.26 * face/unfamiliar/first + 0.13 * face/unfamiliar/immediate + 0.11 * face/unfamiliar/long' (average, N=95), [-0.5, 2] sec, 376 ch, ~9.8 MB>

In [184]:
evoked_scrambled

<Evoked  |  '0.50 * scrambled/first + 0.28 * scrambled/immediate + 0.22 * scrambled/long' (average, N=50), [-0.5, 2] sec, 376 ch, ~9.8 MB>

### Save your figure as pdf

In [185]:
%matplotlib qt
import numpy as np
contrast.plot_topomap(times=np.linspace(0.05, 0.15, 5), ch_type='mag')
plt.savefig('toto.pdf')
!open toto.pdf  # works only on a mac

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Compute the evoked data for 'famous', 'unfamiliar', 'scrambled' faces</li>
      <li>Crop the data between -0.2s and 0.4s</li>     
      <li>Plot the channel EEG065 in all 3 conditions using mne.viz.plot_compare_evokeds function</li>
    </ul>
</div>

See: https://mne.tools/stable/generated/mne.viz.plot_compare_evokeds.html

In [ ]:
evoked_famous = epochs['famous'].average().crop(-0.1, 0.4)
evoked_scrambled = epochs['scrambled'].average().crop(-0.1, 0.4)
evoked_unfamiliar = epochs['unfamiliar'].average().crop(-0.1, 0.4)

In [ ]:
plt.close('all')
mne.viz.plot_evoked_topo([evoked_famous, evoked_scrambled, evoked_unfamiliar]);

In [ ]:
evokeds = {k:epochs[k].average().crop(-0.1, 0.4)
           for k in ['famous', 'unfamiliar', 'scrambled']}

In [ ]:
plt.close('all')
mne.viz.plot_compare_evokeds(evokeds, picks='EEG065');

## ADVANCED: Customize your plots

Want to have every text in blue?

In [188]:
import matplotlib as mpl
fig = evoked1.plot(show=False)  # butterfly plots
fig.subplots_adjust(hspace=1.0)
for text in fig.findobj(mpl.text.Text):
    text.set_fontsize(18)
    text.set_color('blue')
for ax in fig.get_axes():
    ax.axvline(0., color='red', linestyle='--')
plt.tight_layout()
fig.savefig('plot_erf.pdf');